Poisson Equation on a Unit Square with Dirichlet Boundary Conditions Solver

Polar Poisson equation: $u_{rr}+\frac{1}{r}u_r+\frac{1}{r^2}u_{\theta\theta} = -f$ 

In [99]:
import basix.ufl
import ufl

from mpi4py import MPI

import matplotlib.pyplot as plt
import numpy as np
import pyvista


from dolfinx import fem, mesh, plot, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
import dolfinx

import gmsh

gmsh.initialize()
gmsh.model.add('circular_disk')
center = (0,0,0)
R = 1
disk = gmsh.model.occ.addDisk(*center, R, R)
gmsh.model.occ.synchronize()
gmsh.model.addPhysicalGroup(2, [disk], tag=2)
boundary = gmsh.model.getBoundary([(2,disk)],recursive=False, oriented=False)
gmsh.model.addPhysicalGroup(1, [disk], tag=12)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)
gmsh.model.mesh.generate(2)
gmsh.model.mesh.setOrder(3)
circular_mesh, cell_marker, facet_marker = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, MPI.COMM_WORLD, 0, gdim=2)
gmsh.finalize()   

domain = circular_mesh

#--------------------------------------------------------------------#

# All problem inputs must either be a lambda function in x or a real number.

# Input Source Term.
source = 0

# Define Dirchlet boundary condition on the disk
u_omega = 2

# Input an exact solution to provide error estimates or otherwise input None.
exact_u = 2

#--------------------------------------------------------------------#

Info    : Meshing 1D...
Info    : Meshing curve 1 (Ellipse)
Info    : Done meshing 1D (Wall 0.000202526s, CPU 0.000228s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0244425s, CPU 0.024046s)
Info    : 411 nodes 821 elements
Info    : Meshing order 3 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 3
Info    : [ 60%] Meshing surface 1 order 3
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 0.836177
Info    : Done meshing order 3 (Wall 0.00696079s, CPU 0.004613s)


In [100]:
V = fem.functionspace(domain, ("Lagrange", 1))

dofs_omega = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 1))

boundary_values = [u_omega]
dofs = [dofs_omega]
bcs = [ _ ]

for i, boundary_value in enumerate(boundary_values):
    if str(boundary_value).isnumeric():
        boundary_value = default_scalar_type(boundary_value)
        bcs[i] = fem.dirichletbc(boundary_value, dofs[i], V)
    else:
        boundary_function = fem.Function(V)
        boundary_function.interpolate(boundary_value)
        bcs[i] = fem.dirichletbc(boundary_function, dofs[i])

    

In [101]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

if callable(source):
    f = fem.Function(V)
    f.interpolate(source)
else:
    f = fem.Constant(domain, default_scalar_type(0))
x = ufl.SpatialCoordinate(domain)

def grad_polar(g, x):
    return ufl.as_vector([g.dx(0), 1/x[0]*g.dx(1)])

a = ufl.dot(grad_polar(u, x), grad_polar(v, x)) *x[0]* ufl.dx
L = f * v *x[0]* ufl.dx 

problem = fem.petsc.LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [102]:
if exact_u == None:
    pass
else: 
    if callable(exact_u):
        V2 = fem.functionspace(domain, ("Lagrange", 2))
        uex = fem.Function(V2)
        uex.interpolate(exact_u)
        uex_1 = fem.Function(V)
        uex_1.interpolate(uex)
        error_max = np.max(np.abs(uex_1.x.array-uh.x.array)) 
        error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)
    else:
        uex = fem.Constant(domain, default_scalar_type(exact_u))
        uex_1 = exact_u
        error_max = np.max(np.abs([uex_1]*len(uh.x.array)-uh.x.array)) 
        error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)
        
    L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
    error_local = fem.assemble_scalar(L2_error)
    error_L2 = np.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

    # Only print the error on one process
    if domain.comm.rank == 0:
        print(f"Error_L2 : {error_L2:.10e}")
        print(f"Error_max : {error_max:.10e}")

Error_L2 : 6.0157483231e-11
Error_max : 5.2414517171e-11


In [103]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [104]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…